AutoGen - Memory Support

In [23]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.ui import Console
from autogen_core.memory import ListMemory, MemoryContent, MemoryMimeType
from dotenv import load_dotenv

import os

In [24]:
load_dotenv(override=True)

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2023-05-15")
azure_openai_deployment_name = os.getenv(
    "AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-35-turbo")

if not azure_openai_api_key or \
        not azure_openai_endpoint or \
        not azure_openai_api_version or \
        not azure_openai_deployment_name:
    raise ValueError(
        "Please set Azure Open AI Endpoint Details in your environment variables.")


In [25]:
model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=azure_openai_deployment_name,
    model=azure_openai_deployment_name,
    api_key=azure_openai_api_key,
    azure_endpoint=azure_openai_endpoint,
    api_version=azure_openai_api_version,
)

In [26]:
user_memory = ListMemory()

await user_memory.add(
    MemoryContent(
        mime_type=MemoryMimeType.TEXT,
        content="The weather should be in metric units, like Celsius and meters per second.",
    )
)

await user_memory.add(
    MemoryContent(
        mime_type=MemoryMimeType.TEXT,
        content="I like to have always vegan recipes.",
    )
)

In [27]:
def get_weather(city: str, units:str = "imperial") -> str:
    if units == "metric":
        return f"The weather in {city} is 20 degrees Celsius with a wind speed of 5 meters per second."
    elif units == "imperial":
        return f"The weather in {city} is 68 degrees Fahrenheit with a wind speed of 11 miles per hour."
    else:
        return f"Sorry, I don't know the weather in {city} for the units {units}."
    
assistant_agent = AssistantAgent(
    name="WeatherAssistant",
    model_client=model_client,
    memory=[user_memory],
    system_message="You are a helpful assistant that provides weather information.",
    tools=[get_weather],
)

In [28]:
stream = assistant_agent.run_stream(task = "What's the weather in Paris?")

await Console(stream)

---------- TextMessage (user) ----------
What's the weather in Paris?
---------- MemoryQueryEvent (WeatherAssistant) ----------
[MemoryContent(content='The weather should be in metric units, like Celsius and meters per second.', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='I like to have always vegan recipes.', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None)]
---------- ToolCallRequestEvent (WeatherAssistant) ----------
[FunctionCall(id='call_q0qz5AxhPgDUqcdZ8B4LmgGa', arguments='{"city":"Paris","units":"metric"}', name='get_weather')]
---------- ToolCallExecutionEvent (WeatherAssistant) ----------
[FunctionExecutionResult(content='The weather in Paris is 20 degrees Celsius with a wind speed of 5 meters per second.', name='get_weather', call_id='call_q0qz5AxhPgDUqcdZ8B4LmgGa', is_error=False)]
---------- ToolCallSummaryMessage (WeatherAssistant) ----------
The weather in Paris is 20 degrees Celsius with a wind speed of 5 meters per

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 13, 16, 12, 59, 179597, tzinfo=datetime.timezone.utc), content="What's the weather in Paris?", type='TextMessage'), MemoryQueryEvent(source='WeatherAssistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 13, 16, 12, 59, 183605, tzinfo=datetime.timezone.utc), content=[MemoryContent(content='The weather should be in metric units, like Celsius and meters per second.', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='I like to have always vegan recipes.', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None)], type='MemoryQueryEvent'), ToolCallRequestEvent(source='WeatherAssistant', models_usage=RequestUsage(prompt_tokens=116, completion_tokens=19), metadata={}, created_at=datetime.datetime(2025, 6, 13, 16, 13, 0, 610169, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_q0qz5AxhPgDUqcdZ8B4Lm

In [29]:
await assistant_agent._model_context.get_messages()

[UserMessage(content="What's the weather in Paris?", source='user', type='UserMessage'),
 SystemMessage(content='\nRelevant memory content (in chronological order):\n1. The weather should be in metric units, like Celsius and meters per second.\n2. I like to have always vegan recipes.\n', type='SystemMessage'),
 AssistantMessage(content=[FunctionCall(id='call_q0qz5AxhPgDUqcdZ8B4LmgGa', arguments='{"city":"Paris","units":"metric"}', name='get_weather')], thought=None, source='WeatherAssistant', type='AssistantMessage'),
 FunctionExecutionResultMessage(content=[FunctionExecutionResult(content='The weather in Paris is 20 degrees Celsius with a wind speed of 5 meters per second.', name='get_weather', call_id='call_q0qz5AxhPgDUqcdZ8B4LmgGa', is_error=False)], type='FunctionExecutionResultMessage')]

In [30]:
stream = assistant_agent.run_stream(task ="Write brief meal recipe with broth")

await Console(stream)

---------- TextMessage (user) ----------
Write brief meal recipe with broth
---------- MemoryQueryEvent (WeatherAssistant) ----------
[MemoryContent(content='The weather should be in metric units, like Celsius and meters per second.', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='I like to have always vegan recipes.', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None)]
---------- TextMessage (WeatherAssistant) ----------
Here's a simple vegan broth recipe:

### Vegan Vegetable Broth

#### Ingredients:
- 1 tablespoon olive oil
- 1 onion, chopped
- 2 carrots, chopped
- 2 celery stalks, chopped
- 3 garlic cloves, minced
- 1 bay leaf
- 1 teaspoon dried thyme
- 1 teaspoon black peppercorns
- 2 liters water
- Salt to taste
- Fresh herbs (parsley or dill) for garnish

#### Instructions:
1. **Sauté the Vegetables**: In a large pot, heat the olive oil over medium heat. Add the chopped onion, carrots, and celery. Sauté for about 5-7 minutes unti

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 13, 16, 13, 0, 656494, tzinfo=datetime.timezone.utc), content='Write brief meal recipe with broth', type='TextMessage'), MemoryQueryEvent(source='WeatherAssistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 13, 16, 13, 0, 662494, tzinfo=datetime.timezone.utc), content=[MemoryContent(content='The weather should be in metric units, like Celsius and meters per second.', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='I like to have always vegan recipes.', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None)], type='MemoryQueryEvent'), TextMessage(source='WeatherAssistant', models_usage=RequestUsage(prompt_tokens=213, completion_tokens=283), metadata={}, created_at=datetime.datetime(2025, 6, 13, 16, 13, 4, 821745, tzinfo=datetime.timezone.utc), content="Here's a simple vegan broth recipe:\n\n### Vega

In [31]:
from autogen_ext.memory.chromadb import ChromaDBVectorMemory, PersistentChromaDBVectorMemoryConfig
from pathlib import Path

In [32]:
chroma_user_memory = ChromaDBVectorMemory(
    config=PersistentChromaDBVectorMemoryConfig(
        collection_name="user_memory",
        persistent_path=os.path.join(str(Path.home()), ".chroma_autogen"),
        k=2,
        scrore_threshold=0.5,
    )
)

In [33]:
await chroma_user_memory.add(
    MemoryContent(
        mime_type=MemoryMimeType.TEXT,
        content="I like to have always vegan recipes.",
    )
)

await chroma_user_memory.add(
    MemoryContent(
        mime_type=MemoryMimeType.TEXT,
        content="The weather should be in metric units, like Celsius and meters per second"))

In [34]:
assistant_agent_v2 = AssistantAgent(
    name="AssistantV3",
    model_client=model_client,
    memory=[chroma_user_memory],
    system_message="You are a helpful assistant that provides weather information.",
    tools=[get_weather],
)

In [35]:
stream = assistant_agent_v2.run_stream(task = "What's the weather in Paris?")

await Console(stream)

---------- TextMessage (user) ----------
What's the weather in Paris?
---------- MemoryQueryEvent (AssistantV3) ----------
[MemoryContent(content='The weather should be in metric units, like Celsius and meters per second', mime_type='MemoryMimeType.TEXT', metadata={'mime_type': 'MemoryMimeType.TEXT', 'score': 0.48089420795440674, 'id': '12c434d4-ab57-4fce-b348-36d32a6ec0cd'}), MemoryContent(content='The weather should be in metric units, like Celsius and meters per second', mime_type='MemoryMimeType.TEXT', metadata={'mime_type': 'MemoryMimeType.TEXT', 'score': 0.48089420795440674, 'id': 'feb5fecb-8894-4eb2-a546-3e3a1d7c06c1'})]
---------- ToolCallRequestEvent (AssistantV3) ----------
[FunctionCall(id='call_jo368tpSj8sgEzcJJdZYe2aA', arguments='{"city":"Paris","units":"metric"}', name='get_weather')]
---------- ToolCallExecutionEvent (AssistantV3) ----------
[FunctionExecutionResult(content='The weather in Paris is 20 degrees Celsius with a wind speed of 5 meters per second.', name='get

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 13, 16, 13, 5, 816288, tzinfo=datetime.timezone.utc), content="What's the weather in Paris?", type='TextMessage'), MemoryQueryEvent(source='AssistantV3', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 13, 16, 13, 6, 198447, tzinfo=datetime.timezone.utc), content=[MemoryContent(content='The weather should be in metric units, like Celsius and meters per second', mime_type='MemoryMimeType.TEXT', metadata={'mime_type': 'MemoryMimeType.TEXT', 'score': 0.48089420795440674, 'id': '12c434d4-ab57-4fce-b348-36d32a6ec0cd'}), MemoryContent(content='The weather should be in metric units, like Celsius and meters per second', mime_type='MemoryMimeType.TEXT', metadata={'mime_type': 'MemoryMimeType.TEXT', 'score': 0.48089420795440674, 'id': 'feb5fecb-8894-4eb2-a546-3e3a1d7c06c1'})], type='MemoryQueryEvent'), ToolCallRequestEvent(source='AssistantV3', models_usage=

In [36]:
stream = assistant_agent.run_stream(task ="Write brief meal recipe with broth")

await Console(stream)

---------- TextMessage (user) ----------
Write brief meal recipe with broth
---------- MemoryQueryEvent (WeatherAssistant) ----------
[MemoryContent(content='The weather should be in metric units, like Celsius and meters per second.', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='I like to have always vegan recipes.', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None)]
---------- TextMessage (WeatherAssistant) ----------
Here’s a delicious vegan soup recipe that uses broth:

### Vegan Vegetable Soup

#### Ingredients:
- 1 tablespoon olive oil
- 1 onion, diced
- 2 carrots, sliced
- 2 celery stalks, chopped
- 3 garlic cloves, minced
- 1 zucchini, diced
- 1 can (400g) diced tomatoes
- 1 liter vegetable broth
- 1 teaspoon dried oregano
- 1 teaspoon dried basil
- Salt and pepper to taste
- Fresh spinach or kale (optional)
- Lemon juice (optional for garnish)

#### Instructions:
1. **Sauté the Vegetables**: In a large pot, heat the olive oil

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 13, 16, 13, 7, 66615, tzinfo=datetime.timezone.utc), content='Write brief meal recipe with broth', type='TextMessage'), MemoryQueryEvent(source='WeatherAssistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 13, 16, 13, 7, 71182, tzinfo=datetime.timezone.utc), content=[MemoryContent(content='The weather should be in metric units, like Celsius and meters per second.', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='I like to have always vegan recipes.', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None)], type='MemoryQueryEvent'), TextMessage(source='WeatherAssistant', models_usage=RequestUsage(prompt_tokens=549, completion_tokens=307), metadata={}, created_at=datetime.datetime(2025, 6, 13, 16, 13, 12, 128409, tzinfo=datetime.timezone.utc), content='Here’s a delicious vegan soup recipe that uses b

In [37]:
chroma_user_memory.dump_component().model_dump_json()

'{"provider":"autogen_ext.memory.chromadb.ChromaDBVectorMemory","component_type":"memory","version":1,"component_version":1,"description":"Store and retrieve memory using vector similarity search powered by ChromaDB.","label":"ChromaDBVectorMemory","config":{"client_type":"persistent","collection_name":"user_memory","distance_metric":"cosine","k":2,"allow_reset":false,"tenant":"default_tenant","database":"default_database","persistence_path":"./chroma_db"}}'